In [66]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options

In [67]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/curtissmith/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [70]:
# Visit the Beer Advocate Top 250 page
url = 'https://www.beeradvocate.com/beer/popular/'
browser.visit(url)

In [71]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
top_250_home = soup(html, 'html.parser')

In [72]:
# find all a tags
all_a = top_250_home.find_all('a')

# select only relevant link (those with "beer/profile" in link)
links = [x for x in all_a if '/beer/profile/' in str(x)]

# iterate and get links to beers - even indices are links to beers, odd are links to brewery
beer_a_tags = []
for index, item in enumerate(links):
    if index % 2 == 0:
        beer_a_tags.append(item)
        
# get href url for navigating to beer webpage
beer_href = [x.get('href') for x in beer_a_tags]

### Loop Through Links to Retrieve Beer Information

In [73]:
# initialize list of dictionaries
beer_info = []

In [74]:
start_time = time.time()

#iterate through links
for link in beer_href:
    
    # navigate to link
    browser.click_link_by_href(link)
    
    # Parse the beer html with soup
    html = browser.html
    beer_soup = soup(html, 'html.parser')
    
    # Get beer and brewery
    header = beer_soup.find('div', class_='titleBar')
    brewery = header.find("span").text
    beer_name = header.text.replace(brewery, "")
    beer_name = beer_name.replace('\n', "")

    # get ABV and rating
    spans = beer_soup.find_all('span', class_='Tooltip')
    abv = spans[2].text
    rating = spans[6].text

    # get style
    a = beer_soup.find_all('a', class_='Tooltip')
    style = a[4].text

    # get reviews
    review_div = beer_soup.find_all('div', {"id": "rating_fullview_content_2"})
    reviews = [item.text.split("overall: ")[1] for item in review_div]
    
    # create dictionary and add to list
    beer_info.append({"beer_name": beer_name,
                      "beer_style": style,
                      "brewery": brewery,
                      "abv": abv,
                      "rating": rating,
                      "reviews": reviews})
    
    browser.back()
    
    # Parse the beer html with soup
    html = browser.html
    top_250 = soup(html, 'html.parser')
    
length = (time.time() - start_time)
print(f'Script ran in {length} seconds / {length/60} minutes')

/Users/curtissmith/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:477: FutureWarning: browser.find_link_by_href is deprecated. Use browser.links.find_by_href instead.
  FutureWarning,


IndexError: list index out of range

In [75]:
len(beer_info)

245

In [76]:
df = pd.DataFrame(beer_info)

In [77]:
df.head()

,beer_name,beer_style,brewery,abv,rating,reviews
0,Breakfast Stout,Stout - Oatmeal,Founders Brewing Company,8.3%,4.55,"[5I bought this one on a whim. I love stoats, ..."
1,90 Minute IPA,IPA - Imperial,Dogfish Head Craft Brewery,9%,4.3,[4.5I am not a lover of all things IPS. This o...
2,Two Hearted Ale,IPA - American,Bell's Brewery - Eccentric Café & General Store,7%,4.3,"[4I've had this many time over the years, but ..."
3,Pliny The Elder,IPA - Imperial,Russian River Brewing Company,8%,4.69,[5drank while enjoying the Browns v Chiefs gam...
4,Heady Topper,IPA - New England,The Alchemist,8%,4.76,"[4.5Beautiful hazy golden color with thick, wh..."


In [60]:
df.to_csv("top_150.csv", index=False)